<a href="https://colab.research.google.com/github/filsto/3DNN/blob/main/3Dconvolutions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from keras.layers import Conv3D, MaxPool3D, Flatten, Dense
from keras.layers import Dropout, Input, BatchNormalization
from sklearn.metrics import confusion_matrix, accuracy_score
from plotly.offline import iplot, init_notebook_mode
from keras.losses import categorical_crossentropy
from keras.optimizers import Adadelta
import plotly.graph_objs as go
from matplotlib.pyplot import cm
from keras.models import Model
import numpy as np
import keras
import h5py

init_notebook_mode(connected=True)

%matplotlib inline

In [ ]:
with h5py.file('../input/full_dataset_vectors.h5', 'r') as dataset:
  x_train = dataset["X_train"][:]
  x_test = dataset["X_test"][:]
  y_train = dataset["y_train"][:]
  y_test = dataset["y_test"][:]


In [ ]:
#introduce the channel dimension in the input dataset
xtrain = np.ndarray((x_train.shape[0],4096,3))
xtest = np.ndarray((x_test.shape[0],4096,3))

#iterate in train and test, add the rgb dimension
def add_rgb_dimension(array):
  scaler_map=cm.ScalarMappable(cmap="Oranges")
  array=scaler_map.to_rgba(array)[:,:-1]
  return array

for i in range(x_train.shape[0]):
  xtrain[i] = add_rg_dimension(x_train[i])
for i in range(x_test.shape[0]):
  xtest[i] = add_rg_dimension(x_test[i])

#convert to 1 +4D space (1st argument represents the number of rows in the dataset)
xtrain = xtrain.reshape(x_train.shape[0], 16,16,16,3)
xtest = xtest.reshape(x_test.shape[0], 16,16,16,3)

#convert target variable into one-hot
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)


In [ ]:
#input layer
input_layer = Input((16,16,16,3))

#convolutional layers
conv_layer1 = Conv3D(filters=8,kernel_size=(3,3,3), activation ='relu')(input_layer)
conv_layer2 = Conv3D(filters=16,kernel_size=(3,3,3), activation ='relu')(conv_layer1)

#add max pooling to obtain the most informatic features
pooling_layer1 = MaxPool3D(pool_size=(2,2,2))(conv_layer2)

conv_layer3 = Conv3D(filters=32,kernel_size=(3,3,3), activation ='relu')(pooling_layer1)
conv_layer4 = Conv3D(filters=64,kernel_size=(3,3,3), activation ='relu')(conv_layer3)
pooling_layer2 = MaxPool3D(pool_size=(2,2,2))(conv_layer4)

#perform batch nroalization on the convolution outputs before feeding it to MLP architecture
pooling_layer2 = BatchNormalization()(pooling_layer2)
flatten_layer = Flatten()(pooling_layer2)

#create an MLP architecture with dense layers 4096 > 512 >10
#add dropouts to avoid overfitting / perform regularization
dense_layer1 = Dense(units=2048, activation = 'relu')(flatten_layer)
dense_layer1 = Dropout(0.4)(dense_layer1)
dense_layer2 = Dense(units=512, activation = 'relu')(dense_layer1)
dense_layer2 = Dropout(0.4)(dense_layer2)
output_layer = Dense(units=10, activation='softmax')(dense_layer2)

#define the model with input layer and output layer
model = Model(inputs=input_layer, outputs=ouput_layer)

In [ ]:
modele.compile(loss=categorical_crossentropy, optimizer=Adadelta(lr=0.1), metrics=['acc'])
model.fit(x=xtrain, y=y_train, tach_size=128, epochs=50, validation_split=0.2)